# Physics Informed Neural Networks

This notebook provides a set of utilities allowing to compute the power flow under the physics informed graph neural network. We start by explaining the theory behind the DC power flow and how it can be used to model the power grid with a Graph Neural Networks (GNN). 

<span style="color:red">It is important to notice that the dataset used for competition are generated using AC solver and the solution should be based on AC power flow equations. This notebook is only for illustration to show how physics informed GNN could be used to model a simple power grid.</span>

More details concerning the power flow equations could be found in [this article](https://home.engineering.iastate.edu/~jdm/ee458_2011/PowerFlowEquations.pdf).

#### The DC Power flow
Let's study the real power flow expression which could be obtained at a bus $k$:

$P_k = \sum_{j\in N(k), j \neq k} B_{kj}(\theta_k - \theta_j)$

where $B_{kj}$ corresponds to real part of the admittance matrix element connecting nodes $k$ and $j$ (the imaginary part in DC power flow is zero).

However, we require to compute the power flows on the lines (this is usually the information needed by operational and planning engineers as they study the power system).

$P_{kj} = B_{kj}(\theta_k - \theta_j)$

This operation could be generalized to DC power flow computation. We assume that we are given the network with the following information:
- total number of buses is $N$, tital number of branches is $M$;
- Bus number 1 identified as the reference;
- Real power injections at all buses except bus 1;
- Network topology;
- Admittances for all branches.

The DC power flow equations are given in matrix form as: 

$P = B^\prime\theta$ where:
- $\underline{P}$ is the vector of nodal injections for buses $2, ..., N$
- $\underline{\theta}$ is the vector of nodal phase angles for buses $2, ..., N$
- $\underline{B^\prime}$ is a specific form of admittance matrix which requires some explainations concerning its construction: 
    1. Remove the imaginary part `j` from admittance matrix (Y-bus)
    2. Replace the diagonal element $\underline{B^\prime}_{kk}$ with the sum of non-diagonal elements in row $k$.
    3. Multiply all off-diagonals by $-1$.
    4. Remove row 1 and column 1.

Although the above mentioned equation provides the ability to compute the angles, it does not provide the line flows. A systematic method of computing the line flows is:

$P_B = (D \times A) \times \theta$

where:
- $P_B$ is the vector of branch flows. It has dimension of $M \times 1$.
- $\theta$ is (as before) the vector of nodal phase angles for buses $2, ..., N$.
- $D$ is an $M \times M$ matrix have non-diagonal elements of zeros; the diagonal element in position row $k$, column $k$ contains the negative of the susceptance of the $k^{th}$ branch.
- $A$ is the $M \times (N-1)$ node-arc incidence matrix. It is also called adjacency matrix, or the connection matrix. 
    - Element (k,j) of $A$ is 1 if the $k^{th}$ branch begins at node $j$, -1 if the $k^{th}$ branch terminates at node $j$, and 0 otherwise.
    - Note that this matrix has only $N-1$ columns. We do not form a column with the reference bus, in order to conform to the vector $\theta$ which is of dimension $(N-1) \times 1$.

Various functions provided in `graph_utils.py` allows allow to get the phase angles $\theta$ at bus level and compute the branch power flows from phase angles ()`get_active_power`).

### Prerequisites
To be able to use this notebook and the corresponding functions, you should install the `torch-geometric` package

In [ ]:
!pip install pytorch-geometric

In [ ]:
import matplotlib.pyplot as plt
import torch
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from utils.graph_utils_generic import prepare_dataset

In [ ]:
# Use some required pathes
import pathlib
DATA_PATH = pathlib.Path().resolve() / "input_data_local" / "lips_case14_sandbox"
BENCH_CONFIG_PATH = pathlib.Path().resolve() / "configs" / "benchmarks" / "lips_case14_sandbox.ini"

Download the dataset, if it has not been already done. The dataset used for the purpose of this notebook is generated using DC approximation solver and is only for illustration purpose.

In [ ]:
## Download the dataset through the dedicated lips function
from lips.dataset.powergridDataSet import downloadPowergridDataset

downloadPowergridDataset("input_data_local", "Benchmark_DC")

Load the benchmark. Here, we select a benchmark (`Benchmark_DC`) where the data are generated using DC (Direct Current) approximation solver. It is only for illustration purpose and the participants should use the competition benchmark and dataset.

In [ ]:
benchmark = PowerGridBenchmark(benchmark_name="Benchmark_DC",
                               benchmark_path=DATA_PATH,
                               load_data_set=True,
                               log_path=None,
                               config_path=BENCH_CONFIG_PATH,
                              )

In [ ]:
# Verify the dataset sizes
print(benchmark.train_dataset.size)
print(benchmark.val_dataset.size)
print(benchmark._test_dataset.size)
print(benchmark._test_ood_topo_dataset.size)

### Prepare the dataset

In order to use a Physics Informed Graph Neural Network, the dataset should be prepared and structured correctly. We provide a set of functions in `utils/graph_utils.py` which help to structure the data as graph and to create corresponding data loaders. This is based on pytorch geometric dependence which should be installed to be able to use the following commands. 

To install pytorch geometric library, you could use following command.
```command
! pip install torch-geometric
```

By calling the `prepare_dataset` function, a set of functions will be called automatically to create the final data loaders for each of `train`, `validation`, `test` and `test_ood` datasets.

In [ ]:
device = torch.device("cpu") # or "cuda:0" if you have any GPU
train_loader, val_loader, test_loader, test_ood_loader = prepare_dataset(benchmark=benchmark, batch_size=512, device=device)

Lets have a closer look at a batch of data:

In [ ]:
batch = next(iter(test_loader))

In [ ]:
batch

As we can see here, a batch is composed of following elements (some of the elements could not be used by the model):

- ``x`` : is the set of features at the nodes of the graph (`prod_p` and `load_p` in this specific case);
- ``edge_index`` : is the pytorch geometric way to create the connections among nodes (works as an adjacency matrix).;
- ``edge_attributes`` : the attributes on each edge connecting a pair of nodes (admittance values);
- ``y`` : corresponds to the targets at each node of the graph which should be predicted by the model (in this case the target is only the voltage angles from which the active powers could be inferred);
- ``edge_index_no_diag`` : is same as edge_index by removing the diagonal elements.
- ``edge_attr_no_diag`` : is same as edge_attr by removing the diagonal elements.
- ``ybus`` : is the admittance matrix in its classic form.
- ``batch`` and ``ptr`` are pytorch geometric specific parameters. 

## Create a physics informed GNN
This GNN is not trainable. Each layer can be seen as one step optimization of power flow equation.

In [ ]:
import torch
from utils.graph_utils_generic import GPGmodel_without_NN

device = torch.device("cpu") # select "cuda:0" if you have a GPU
gpg_model_wo_nn = GPGmodel_without_NN(ref_node=0, num_gnn_layers=100, device=device)
gpg_model_wo_nn.to(device)

As can be seen, this architecture is composed of different layers:
- `GPGinput_without_NN`: The input layer of Physics informed GNN
- `LocalConservationLayer`:  The layer that computes the local conservation error after each layer.
- `GPGintermediate`: The intermediate layer of GNN that updates the theta values with respect ot 

#### Input layer
It performs the computation through the local conservation equation at a node i
$P^i_{prod} - P^i_{load} = \sum_{j \in \{i, N(i)\}} \theta_j \times y_{ij}$
where:
- $P^i_{prod}$: Active power generated at the node $i$
- $P^i_{load}$: Active power consumed at the node $i$
- $N(i)$ : the neighborhood at node $i$
- $\theta_j$: the voltage angle at node $j$
- $y_{ij}$: the admittance matrix element (power line) between node i and j 
    
From this equation, we can update the theta values as follows:
$\hat{\theta}_i = \frac{P^i_{prod} - P^i_{load} - \sum_{j \in N(i), i\neq j} \theta_j \times y_{ij}}{y_{ii}}$

The $\theta_j$ for this first layer is initialized with zeros.

#### Intermediate layer
Once the $\hat{\theta}_i$ is estimated for all the nodes in power grid, we pass it as an argument to the intermediate layer. This layer performs exactly the same operation as input layer by replacing the $\theta$ with current estimations in previous layer.

#### Local conservation layer
It computes the local conservation error after each layer in the Physics Informed GNN to trace the error as:

$Error = P^i_{prod} - P^i_{load} - \sum_{j \in \{i, N(i)\}} \theta_j \times y_{ij}$


In [ ]:
import numpy as np

predictions = []
observations = []
error_per_batch = []
for batch in test_loader:
    out, errors = gpg_model_wo_nn(batch)
    predictions.append(out)
    observations.append(batch.y)
    error_per_batch.append([float(error.detach().cpu().numpy()) for error in errors])
observations = torch.vstack(observations)
predictions = torch.vstack(predictions)
errors = np.vstack(error_per_batch)
errors = errors.mean(axis=0)

Visualize the error through the iterations (graph layers)

In [ ]:
plt.figure()
plt.plot(errors)
plt.xlabel("# GNN layers (iterations)")
plt.ylabel("Local Conservation Error")

Convert the angles from Radian to Degree ($\times \frac{180}{\pi}$)

In [ ]:
import math
predictions = predictions * (180/math.pi)

In [ ]:
np.concatenate((predictions, observations), axis=1)

Compute the MAPE metric on voltage angles

In [ ]:
MAPE = abs((observations - (predictions)) / (observations + 1e-10)).mean()
print("MAPE: ", MAPE)

Compute MAPE10 on voltage angles (90% highest quantile of the distribution).

In [ ]:
from lips.metrics.ml_metrics.external_metrics import mape_quantile

MAPE_10 = mape_quantile(y_true=observations.detach().cpu(), y_pred=predictions.detach().cpu(), quantile=0.9)
print("MAPE 10: ", MAPE_10)

Once the voltage angles are computed using the model, we can transform them to power flows (active powers) using the provided function `get_all_active_powers`.

<span style="color:red">In this notebook, we are not interested in other power flow variables such as voltages and currents. However, for the competition purpose, these additional variables should also be predicted by the solutions.</span>

In [ ]:
import warnings
from utils.graph_utils_generic import get_obs, get_all_active_powers

warnings.filterwarnings("ignore")
env, obs = get_obs(benchmark)
p_ors_pred, p_exs_pred = get_all_active_powers(benchmark._test_dataset.data,
                                               obs,
                                               theta_bus=predictions.view(-1,14).cpu())

In [ ]:
# put the predictions in a dictionary in order to that the LIPS framework could read them for evaluation
my_predictions = {}
my_predictions["p_or"] = p_ors_pred
my_predictions["p_ex"] = p_exs_pred

In [ ]:
MAPE10_Power = mape_quantile(y_true=benchmark._test_dataset.data["p_or"], 
                             y_pred=my_predictions["p_or"], 
                             quantile=0.9)
print("MAPE10 on Active Powers: ", MAPE10_Power)

### Physics law verification 

Once the power flow is computed from the predicted voltage angles, we can verify the local conservation law using the provided function in LIPS framework. This law is one among 8 physics laws that should be respected in this challenge. 

In [ ]:
from lips.metrics.power_grid.local_conservation import local_conservation

LC_tolerance = 1e-2

verification = local_conservation(predictions=my_predictions,
                                  observations=benchmark._test_dataset.data,
                                  tolerance=LC_tolerance,
                                  env=env,
                                  result_level=2)
print(f"Violation percentage: {verification['violation_percentage']:.2f} %")

## Using IDF environment

### Generate some data using DC approximation
In this configuration and for illustration purpose, we consider DC approximation for data generation and we use a grid configuration with only power line disconnection and without any topology reconfiguration. It should be noted that the dataset considered for the competition is generated using AC solver (complex admittance matrix values) and includes topology changes in the grid. 

The topology reconfiguration consists in changing how the nodes of the network are interconnected through two bus bars connected to each node.

In [ ]:
# Use some required pathes
import pathlib
DATA_PATH = pathlib.Path().resolve() / "input_data_local"
BENCH_CONFIG_PATH = pathlib.Path().resolve() / "configs" / "benchmarks" / "lips_idf_2023.ini"

In [ ]:
from lips.benchmark.powergridBenchmark import PowerGridBenchmark

benchmark_idf = PowerGridBenchmark(benchmark_path=DATA_PATH,
                                   benchmark_name="Benchmark_DC",
                                   load_data_set=False, # we set False, as data are not yet generated
                                   config_path=BENCH_CONFIG_PATH,
                                   log_path=None)

Here, we generate 2000 points per dataset for IDF environment. We select following parameters for generation:

- `do_store_physics=True`: Indicate that we want save the physics related variables (`YBus`, `SBus`, etc.)
- `store_as_sparse = True`: Allows to store huge `YBus` Matrix in sparse mode.
- `is_dc=True`: Indicates that we want generate some data using DC approximation. It is selected here only for illustration purpose and because the electricity equations are easier to compute in the case of DC approximation, in comparison to AC. See the first part of this notebook for more explaination concerning the equations.

In [ ]:
benchmark_idf.generate(nb_sample_train=int(2e3),
                       nb_sample_val=int(2e3),
                       nb_sample_test=int(2e3),
                       nb_sample_test_ood_topo=int(2e3),
                       store_as_sparse=True, # NOTE: Specific to envs with 118 or more nodes
                       do_store_physics=True,
                       is_dc=True
                      )

If the data is already generated, load the generated dataset by setting `load_data_set=True`.

In [ ]:
benchmark_idf = PowerGridBenchmark(benchmark_path=DATA_PATH,
                                   benchmark_name="Benchmark_DC",
                                   load_data_set=True,
                                   config_path=BENCH_CONFIG_PATH,
                                   load_ybus_as_sparse=True,
                                   log_path=None)

In [ ]:
print(benchmark_idf.train_dataset.size)

### Prepare the dataset

Prepare the dataset and reorganize them for Graph Neural Networks. In this configuration, the features are active powers for productions and loads and the target is the voltage angle at each node.

In [ ]:
device = torch.device("cpu") # or "cuda:0" if you have any GPU
train_loader, val_loader, test_loader, test_ood_loader = prepare_dataset(benchmark=benchmark_idf, batch_size=512, device=device)

In [ ]:
batch = next(iter(train_loader))

In [ ]:
batch

### Model using Graph Neural Networks

Import the Graph Neural Network which should be used to model the data. In this configuration, our GNN is only based on resolution of power grid equations and no learnable parameter is considered. Feel free to add a learning component, for example to initialize the first theta values at the input layer.

In [ ]:
import torch
from utils.graph_utils_generic import GPGmodel_without_NN

device = torch.device("cpu") # select "cuda:0" if you have a GPU
gpg_model_wo_nn = GPGmodel_without_NN(ref_node=68, num_gnn_layers=2000, device=device)
gpg_model_wo_nn.to(device)

Iterate over the layers of the GNN (Optimization).

In [ ]:
import numpy as np

predictions = []
observations = []
error_per_batch = []
for batch in test_loader:
    out, errors = gpg_model_wo_nn(batch)
    predictions.append(out)
    observations.append(batch.y)
    error_per_batch.append([float(error.detach().cpu().numpy()) for error in errors])
observations = torch.vstack(observations)
predictions = torch.vstack(predictions)
errors = np.vstack(error_per_batch)
errors = errors.mean(axis=0)

In [ ]:
plt.figure()
plt.plot(errors)
plt.xlabel("# GNN layers (iterations)")
plt.ylabel("Local Conservation Error")

Convert the predictions

In [ ]:
import math
predictions = predictions * (180/math.pi)

Evaluate the voltage angles

In [ ]:
MAPE = abs((observations - (predictions*100)) / (observations + 1e-10)).mean()
print("MAPE: ", MAPE)

In [ ]:
from lips.metrics.ml_metrics.external_metrics import mape_quantile

MAPE_10 = mape_quantile(y_true=observations.detach().cpu(), y_pred=predictions.detach().cpu()*100, quantile=0.9)
print("MAPE 10: ", MAPE_10)

Convert the voltage angles at substations (graph nodes) to active powers at power lines.

In [ ]:
import warnings
from utils.graph_utils_generic import get_obs, get_all_active_powers

warnings.filterwarnings("ignore")
env, obs = get_obs(benchmark_idf)
p_ors_pred, p_exs_pred = get_all_active_powers(benchmark_idf._test_dataset.data,
                                               obs,
                                               theta_bus=predictions.view(-1,obs.n_sub).cpu())

In [ ]:
# put the predictions in a dictionary in order to that the LIPS framework could read them for evaluation
my_predictions = {}
my_predictions["p_or"] = p_ors_pred
my_predictions["p_ex"] = p_exs_pred

Evaluate the accuracy of converted active powers

In [ ]:
MAPE10_Power = mape_quantile(y_true=benchmark_idf._test_dataset.data["p_or"], 
                             y_pred=my_predictions["p_or"],
                             quantile=0.9)
print("MAPE10 on Active Powers: ", MAPE10_Power)

Evaluate the physics compliance considering the local conservation criteria

In [ ]:
from lips.metrics.power_grid.local_conservation import local_conservation

LC_tolerance = 1e-2

verification = local_conservation(predictions=my_predictions,
                                  observations=benchmark_idf._test_dataset.data,
                                  tolerance=LC_tolerance,
                                  env=env,
                                  result_level=2)
print(f"Violation percentage: {verification['violation_percentage']:.2f} %")